# Imports

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import gzip
import shutil

# Make sure we can see the full scale of the data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_csv('dirty_df.csv')
df.head()

,Unnamed: 0,cast,crew,id,title,genres,budget,overview,popularity,production_companies,production_countries,revenue,runtime,vote_average,vote_count,keywords,release_date
0,0,TomHanks TimAllen DonRickles JimVarney Wallace...,JohnLasseter JossWhedon AndrewStanton JoelCohe...,862,Toy Story,Animation Comedy Family,30000000,"Led by Woody, Andy's toys live happily in his ...",21.946943,PixarAnimationStudios,UnitedStatesofAmerica,373554033.0,81.0,7.7,5415.0,jealousy toy boy friendship friends rivalry bo...,1995-10-30
1,1,RobinWilliams JonathanHyde KirstenDunst Bradle...,LarryJ.Franco JonathanHensleigh JamesHorner Jo...,8844,Jumanji,Adventure Fantasy Family,65000000,When siblings Judy and Peter discover an encha...,17.015539,TriStarPictures TeitlerFilm InterscopeCommunic...,UnitedStatesofAmerica,262797249.0,104.0,6.9,2413.0,board game disappearance based on children's b...,1995-12-15
2,2,WhitneyHouston AngelaBassett LorettaDevine Lel...,ForestWhitaker RonaldBass RonaldBass EzraSwerd...,31357,Waiting to Exhale,Comedy Drama Romance,16000000,"Cheated on, mistreated and stepped on, the wom...",3.859495,TwentiethCenturyFoxFilmCorporation,UnitedStatesofAmerica,81452156.0,127.0,6.1,34.0,based on novel interracial relationship single...,1995-12-22
3,3,SteveMartin DianeKeaton MartinShort KimberlyWi...,AlanSilvestri ElliotDavis NancyMeyers NancyMey...,11862,Father of the Bride Part II,Comedy,0,Just when George Banks has recovered from his ...,8.387519,SandollarProductions TouchstonePictures,UnitedStatesofAmerica,76578911.0,106.0,5.7,173.0,baby midlife crisis confidence aging daughter ...,1995-02-10
4,4,AlPacino RobertDeNiro ValKilmer JonVoight TomS...,MichaelMann MichaelMann ArtLinson MichaelMann ...,949,Heat,Action Crime Drama Thriller,60000000,"Obsessive master thief, Neil McCauley leads a ...",17.924927,RegencyEnterprises ForwardPass WarnerBros.,UnitedStatesofAmerica,187436818.0,170.0,7.7,1886.0,robbery detective bank obsession chase shootin...,1995-12-15


In [19]:
movie_df.id.value_counts()

141971        3
168538        2
25541         2
15028         2
11115         2
84198         2
13209         2
77221         2
152795        2
12600         2
10991         2
42495         2
14788         2
18440         2
105045        2
159849        2
22649         2
4912          2
132641        2
110428        2
109962        2
119916        2
23305         2
99080         2
97995         2
69234         2
5511          2
265189        2
298721        2
53210         1
18589         1
58333         1
223249        1
288980        1
206197        1
89072         1
270886        1
295595        1
52721         1
271714        1
315850        1
38248         1
122525        1
69352         1
141884        1
86541         1
250556        1
212481        1
217787        1
16247         1
99904         1
307931        1
285851        1
109463        1
289716        1
116439        1
296206        1
150540        1
222517        1
286521        1
107308        1
862           1
59871   

In [11]:
movie_df = pd.read_csv('movies_metadata.csv')
movie_df.head()

/usr/local/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
keywords_df = pd.read_csv('keywords.csv')
keywords_df.head(5)

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [16]:
keywords_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


In [10]:
credits_df = pd.read_csv('credits.csv')
credits_df.head(5)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [5]:
links_df = pd.read_csv('links.csv')
links_df.head(5)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [6]:
ratings_df = pd.read_csv('ratings.csv')
ratings_df.head(5)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [7]:
ratings_df.rename(columns = {'movieId':'id'}, inplace=True)

In [15]:
pd.merge(movie_df, keywords_df, on = "id", how = "left")

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat